In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [6]:
#FINAL PROJECT
#JULIAN FONG AND HAOHUA TANG

#Understanding Layers of Batch Normalization through Convolutional Neural Networks

import os
import numpy as np
import cv2
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras.models import Sequential
from keras import backend as K
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.datasets import cifar10
#%cd /content/gdrive/MyDrive/data
data_path = "/content/gdrive/MyDrive/data"

IMAGE_SIZE = 64

In [7]:
#CODE WAS REFERENCED FROM CODEBASE IN ORDER TO CONSTRUCT FACE DATASET. REFERENCE LINK: https://github.com/Hironsan/BossSensor
def resize_with_pad(image, height=IMAGE_SIZE, width=IMAGE_SIZE):

    def get_padding_size(image):
        h, w, _ = image.shape
        longest_edge = max(h, w)
        top, bottom, left, right = (0, 0, 0, 0)
        if h < longest_edge:
            dh = longest_edge - h
            top = dh // 2
            bottom = dh - top
        elif w < longest_edge:
            dw = longest_edge - w
            left = dw // 2
            right = dw - left
        else:
            pass
        return top, bottom, left, right

    top, bottom, left, right = get_padding_size(image)
    BLACK = [0, 0, 0]
    constant = cv2.copyMakeBorder(image, top , bottom, left, right, cv2.BORDER_CONSTANT, value=BLACK)

    resized_image = cv2.resize(constant, (height, width))

    return resized_image


In [8]:
##CODE WAS REFERENCED FROM CODEBASE IN ORDER TO CONSTRUCT FACE DATASET. REFERENCE LINK: https://github.com/Hironsan/BossSensor
def traverse_dir(path, images, labels):
    for file_or_dir in os.listdir(path):
        abs_path = os.path.abspath(os.path.join(path, file_or_dir))
        print(abs_path)
        if os.path.isdir(abs_path):  # dir
            traverse_dir(abs_path, images, labels)
        else:                        # file
            if file_or_dir.endswith('.jpg'):
                image = read_image(abs_path)
                images.append(image)
                labels.append(path)
    return images, labels

In [9]:
##CODE WAS REFERENCED FROM CODEBASE IN ORDER TO CONSTRUCT FACE DATASET. REFERENCE LINK: https://github.com/Hironsan/BossSensor
def read_image(file_path):
    image = cv2.imread(file_path)
    image = resize_with_pad(image, IMAGE_SIZE, IMAGE_SIZE)

    return image

In [10]:
##CODE WAS REFERENCED FROM CODEBASE IN ORDER TO CONSTRUCT FACE DATASET. REFERENCE LINK: https://github.com/Hironsan/BossSensor
def extract_data(path, images, labels):
    images, labels = traverse_dir(path, images, labels)
    images = np.array(images)
    int_labels = np.zeros(len(images))
    num_classes = 0
    for i in range(1, len(labels)):
        if labels[i][-2:] != labels[i-1][-2:]:
            num_classes += 1
        int_labels[i] = num_classes

    return images, int_labels

In [11]:
##CODE WAS REFERENCED FROM CODEBASE IN ORDER TO CONSTRUCT FACE DATASET. REFERENCE LINK: https://github.com/Hironsan/BossSensor
images = []
labels = []


def load_split_data(images, labels, nb_classes=3):
    images, labels = extract_data(data_path, images, labels)
    labels = np.reshape(labels, [-1])
    img_rows, img_cols = IMAGE_SIZE, IMAGE_SIZE
    X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=1)

    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1)

    if K.image_data_format() == 'channels first':
        X_train = X_train.reshape(X_train.shape[0], 3, img_rows, img_cols)
        X_val = X_val.reshape(X_val.shape[0], 3, img_rows, img_cols)
        X_test = X_test.reshape(X_test.shape[0], 3, img_rows, img_cols)
        input_shape = (3, img_rows, img_cols)
    else:
        X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 3)
        X_val = X_val.reshape(X_val.shape[0], img_rows, img_cols, 3)
        X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 3)
        input_shape = (img_rows, img_cols, 3)

    print('X_train shape:', X_train.shape)
    print(X_train.shape[0], 'train samples')
    print(X_val.shape[0], 'valid samples')
    print(X_test.shape[0], 'test samples')

    y_train = np_utils.to_categorical(y_train, nb_classes)
    y_val = np_utils.to_categorical(y_val, nb_classes)
    y_test = np_utils.to_categorical(y_test, nb_classes)

    X_train = X_train.astype('float32')
    X_val = X_val.astype('float32')
    X_test = X_test.astype('float32')
    X_train /= 255
    X_val /= 255
    X_test /= 255

    return X_train, X_val, X_test, y_train, y_val, y_test, input_shape

In [12]:
X_train, X_val, X_test, y_train, y_val, y_test, input_shape = load_split_data(images, labels)

/content/gdrive/MyDrive/data/freezing
/content/gdrive/MyDrive/data/freezing/991.jpg
/content/gdrive/MyDrive/data/freezing/988.jpg
/content/gdrive/MyDrive/data/freezing/996.jpg
/content/gdrive/MyDrive/data/freezing/999.jpg
/content/gdrive/MyDrive/data/freezing/997.jpg
/content/gdrive/MyDrive/data/freezing/987.jpg
/content/gdrive/MyDrive/data/freezing/992.jpg
/content/gdrive/MyDrive/data/freezing/993.jpg
/content/gdrive/MyDrive/data/freezing/986.jpg
/content/gdrive/MyDrive/data/freezing/990.jpg
/content/gdrive/MyDrive/data/freezing/995.jpg
/content/gdrive/MyDrive/data/freezing/929.jpg
/content/gdrive/MyDrive/data/freezing/972.jpg
/content/gdrive/MyDrive/data/freezing/940.jpg
/content/gdrive/MyDrive/data/freezing/941.jpg
/content/gdrive/MyDrive/data/freezing/994.jpg
/content/gdrive/MyDrive/data/freezing/967.jpg
/content/gdrive/MyDrive/data/freezing/955.jpg
/content/gdrive/MyDrive/data/freezing/921.jpg
/content/gdrive/MyDrive/data/freezing/957.jpg
/content/gdrive/MyDrive/data/freezing/950.

In [13]:
(trainX, trainY), (testX, testY) = cifar10.load_data()

trainY = np_utils.to_categorical(trainY)
testY = np_utils.to_categorical(testY)

trainX = trainX.astype('float32')
trainY = trainY.astype('float32')
testX = testX.astype('float32')
testY = testY.astype('float32')

trainX /= 255
testX /= 255

170500096/170498071 [==============================] - 2s 0us/step


In [14]:
print(X_train.shape)
print(trainX.shape)


(1801, 64, 64, 3)
(50000, 32, 32, 3)


In [45]:
#CODE REFERENCED FROM KERAS TUTORIALS. CODE REFERENCED LINKS: https://keras.io/api/models/model_training_apis/ , https://keras.io/api/preprocessing/image/, https://appliedmachinelearning.blog/2018/03/24/achieving-90-accuracy-in-object-recognition-task-on-cifar-10-dataset-with-keras-convolutional-neural-networks/
#SPECIFICALLY FOR THE TRAIN FUNCTIONS AND THE IMAGE GENERATORS
#CODE FOR CONVOLUTIONAL MODEL WAS CHANGED TO REFLECT GOALS AND AIMS FOR TASK.
class conv_net(object):
    def __init__(self, input_shape, use_batchnorm=True, nb_classes = 3, lr=0.01, BN_selection = [False, False, False, False]):
        self.use_batchnorm = use_batchnorm
        self.lr = lr
        self.model = Sequential()

        self.model.add(Conv2D(64, (3, 3), padding='same', input_shape=input_shape))
        self.model.add(Activation('relu'))
        self.model.add(MaxPooling2D(pool_size=(2, 2), padding = 'same'))
        if use_batchnorm and BN_selection[0]:
            self.model.add(BatchNormalization())
        self.model.add(Conv2D(64, (3, 3)))
        self.model.add(Activation('relu'))
        if use_batchnorm and BN_selection[1]:
            self.model.add(BatchNormalization())
        self.model.add(MaxPooling2D(pool_size=(2, 2), padding = 'same'))

        self.model.add(Conv2D(128, (3, 3), padding='same'))
        self.model.add(Activation('relu'))
        self.model.add(MaxPooling2D(pool_size=(2, 2), padding = 'same'))
        if use_batchnorm and BN_selection[2]:
            self.model.add(BatchNormalization())
        self.model.add(Conv2D(128, (3, 3)))
        self.model.add(Activation('relu'))
        if use_batchnorm and BN_selection[3]:
            self.model.add(BatchNormalization())
        self.model.add(MaxPooling2D(pool_size=(2, 2), padding = 'same'))

        self.model.add(Flatten())
        self.model.add(Dense(512))
        self.model.add(Activation('relu'))
        self.model.add(Dense(nb_classes))
        self.model.add(Activation('softmax'))

    def train(self, X_train, X_val, y_train, y_val, batch_size=32, nb_epoch=40, data_augmentation=True):
        sgd = SGD(lr=self.lr, decay=1e-6, momentum=0.9, nesterov=True)
        self.model.compile(loss='categorical_crossentropy',
                           optimizer=sgd,
                           metrics=['accuracy'])
        if not data_augmentation:
            print('Not using data augmentation.')
            self.model.fit(X_train, y_train,
                           batch_size=batch_size,
                           nb_epoch=nb_epoch,
                           validation_data=(X_val, y_val),
                           shuffle=True)
        else:
            print('Using real-time data augmentation.')

            datagen = ImageDataGenerator(
                featurewise_center=False,  
                samplewise_center=False,  
                featurewise_std_normalization=False,  
                samplewise_std_normalization=False,  
                zca_whitening=False,  
                rotation_range=20, 
                width_shift_range=0.2,  
                height_shift_range=0.2,  
                horizontal_flip=True,  
                vertical_flip=False)  


            datagen.fit(X_train)

            self.model.fit(datagen.flow(X_train, y_train,
                                                  batch_size=batch_size),
                                     steps_per_epoch=(X_train.shape[0])//batch_size,
                                     epochs=nb_epoch,
                                     validation_data=(X_val, y_val))


    def train2(self, trainX, trainY,batch_size=32, nb_epoch=40, data_augmentation=True):
        sgd = SGD(lr=self.lr, decay=1e-6, momentum=0.9, nesterov=True)
        self.model.compile(loss='categorical_crossentropy',
                           optimizer=sgd,
                           metrics=['accuracy'])
        if not data_augmentation:
            print('Not using data augmentation.')
            self.model.fit(trainX, trainY,
                           batch_size=batch_size,
                           epochs=nb_epoch,
                           validation_data=None,
                           shuffle=True)
        else:
            print('Using real-time data augmentation.')

            datagen = ImageDataGenerator(
                featurewise_center=False,  
                samplewise_center=False,  
                featurewise_std_normalization=False,  
                samplewise_std_normalization=False,  
                zca_whitening=False,  
                rotation_range=20,  
                width_shift_range=0.2,  
                height_shift_range=0.2,  
                horizontal_flip=True,  
                vertical_flip=False)  


            datagen.fit(trainX)

            self.model.fit(datagen.flow(trainX, trainY,batch_size=batch_size),steps_per_epoch=(trainX.shape[0])//batch_size,epochs=nb_epoch)
    def evaluate(self, X_test, y_test):
        score = self.model.evaluate(X_test, y_test, verbose=0)
        print("%s: %.2f%%" % (self.model.metrics_names[1], score[1] * 100))


In [ ]:
model = conv_net(input_shape, nb_classes = 3)
model.train(X_train, X_val, y_train, y_val)
model.evaluate(X_test, y_test)




input_size = (32, 32, 3)
model2 = conv_net(input_size, nb_classes = 10)
model2.train2(trainX, trainY)
model2.evaluate(testX, testY)

Using real-time data augmentation.
Epoch 1/40
56/56 [==============================] - 23s 400ms/step - loss: 1.0268 - accuracy: 0.4656 - val_loss: 0.2940 - val_accuracy: 0.9168
Epoch 2/40
56/56 [==============================] - 22s 392ms/step - loss: 0.4650 - accuracy: 0.8303 - val_loss: 0.2245 - val_accuracy: 0.9468
Epoch 3/40
56/56 [==============================] - 22s 394ms/step - loss: 0.2449 - accuracy: 0.9229 - val_loss: 0.0698 - val_accuracy: 0.9834
Epoch 4/40
56/56 [==============================] - 22s 394ms/step - loss: 0.1286 - accuracy: 0.9593 - val_loss: 0.0472 - val_accuracy: 0.9884
Epoch 5/40
56/56 [==============================] - 22s 393ms/step - loss: 0.1418 - accuracy: 0.9591 - val_loss: 0.0606 - val_accuracy: 0.9834
Epoch 6/40
56/56 [==============================] - 22s 394ms/step - loss: 0.1105 - accuracy: 0.9670 - val_loss: 0.0360 - val_accuracy: 0.9900
Epoch 7/40
56/56 [==============================] - 22s 393ms/step - loss: 0.0799 - accuracy: 0.9667 - val_